In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.4 MB/s eta 0:00:00


In [2]:
%env WANDB_PROJECT = final_model_
excluded_session = 3

env: WANDB_PROJECT=final_model_


In [4]:
!pip install wandb

In [5]:
!pip install --upgrade datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [6]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [7]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Speech_dataset", use_auth_token=True)

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating Session1 split:   0%|          | 0/1085 [00:00<?, ? examples/s]

Generating Session2 split:   0%|          | 0/1023 [00:00<?, ? examples/s]

Generating Session3 split:   0%|          | 0/1151 [00:00<?, ? examples/s]

Generating Session4 split:   0%|          | 0/1031 [00:00<?, ? examples/s]

Generating Session5 split:   0%|          | 0/1241 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/minoosh___parquet/minoosh--IEMOCAP_Speech_dataset-3af8fbe4473265f6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
dataset=dataset.remove_columns(['TURN_NAME'])

In [11]:
from datasets import load_metric

metric = load_metric("accuracy")

/tmp/ipykernel_28/1780215247.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [12]:
labels = dataset["Session1"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["2"]

'neu'

In [13]:
model_checkpoint = "facebook/wav2vec2-base"

In [14]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [15]:
max_duration = 10.0

In [16]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [17]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Map:   0%|          | 0/1023 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

Map:   0%|          | 0/1031 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

In [18]:
encoded_dataset = encoded_dataset.rename_column("emotion", "label")

In [19]:
ds1 = encoded_dataset['Session1'].train_test_split(test_size=0.2)
ds2 = encoded_dataset['Session2'].train_test_split(test_size=0.2)
ds3 = encoded_dataset['Session3'].train_test_split(test_size=0.2)
ds4 = encoded_dataset['Session4'].train_test_split(test_size=0.2)
ds5 = encoded_dataset['Session5'].train_test_split(test_size=0.2)

In [20]:
import torch
train_ds = torch.utils.data.ConcatDataset([ds1['train'], 
                                           ds2['train'],
                                           #ds3['train'],
                                           ds4['train'],
                                           ds5['train']])

val_ds = torch.utils.data.ConcatDataset([ds1['test'],
                                         ds2['test'],
                                         #ds3['test'],
                                         ds4['test'],
                                         ds5['test']])

In [21]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.weight', 'classifier.bias', 'classifier

In [22]:
model_name = "wav2vec2.0-base"
batch_size = 8
args = TrainingArguments(
    output_dir="finetuned_wav2vec2.0-base-on-IEMOCAP_3",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to='wandb',
    run_name='finetune_wav2vec2.0-base-on-IEMOCAP_3'
)

In [23]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_3 into local empty directory.


In [25]:
import torch
torch.cuda.empty_cache()

In [26]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
0,1.197000,1.151665,0.473804
2,0.931200,0.972166,0.577449
2,0.818300,0.832219,0.664009
4,0.745600,0.720130,0.719818
4,0.599000,0.785331,0.711845
6,0.416600,0.635892,0.771071
6,0.250700,0.784526,0.741458
8,0.279400,0.818783,0.749431
8,0.177700,0.740591,0.794989
10,0.134400,0.873524,0.761959


TrainOutput(global_step=1635, training_loss=0.39982388274294156, metrics={'train_runtime': 8773.6393, 'train_samples_per_second': 5.987, 'train_steps_per_second': 0.186, 'total_flos': 4.101406126958853e+18, 'train_loss': 0.39982388274294156, 'epoch': 14.93})

In [27]:
trainer.evaluate(encoded_dataset['Session3'])

{'eval_loss': 1.311862826347351,
 'eval_accuracy': 0.6411815812337098,
 'eval_runtime': 83.5553,
 'eval_samples_per_second': 13.775,
 'eval_steps_per_second': 1.723,
 'epoch': 14.93}

In [28]:
trainer.push_to_hub()

To https://huggingface.co/minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_3
   37227af..f343fdb  main -> main

To https://huggingface.co/minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_3
   f343fdb..9334c22  main -> main



'https://huggingface.co/minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_3/commit/f343fdb39aa6a1e19fb9cde794090e4e40f5cd2a'